In [1]:
import os
import re
import sys
import time
import implicit
import numpy as np
import tensorflow as tf
import pandas as pd
from scipy import sparse
from tqdm import tqdm_notebook as tqdm

In [2]:
IS_LOCAL = False
HOME_DIR = '/mnt/E/Projects/Content-based-Neural-Recommender-Systems/' if IS_LOCAL else '../../'
os.environ['HOME_DIR'] = HOME_DIR

sys.path.append("../..")
from utils.prepare_data import get_zen_data, zen_text_iterator

DATA_DIR = f'{HOME_DIR}data/zen/'
WORKING_DIR = f'{HOME_DIR}/models/mlp/'
os.chdir(WORKING_DIR)

In [3]:
!pwd

/data/home/Xetd71/Content-based-Neural-Recommender-Systems/models/mlp


## Load data

In [4]:
items_df, (train_df, test_df) = get_zen_data()

loading items: 328050it [00:32, 10096.95it/s]
loading users: 42977it [02:42, 264.62it/s]


In [5]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [6]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [7]:
# 5
doc2vec_model = Doc2Vec(documents=zen_text_iterator(), vector_size=64, window=10, workers=4, epochs=7)

2019-04-08 07:31:27,523 : INFO : collecting all words and their counts
2019-04-08 07:31:27,524 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2019-04-08 07:31:30,546 : INFO : PROGRESS: at example #10000, processed 2098645 words (694772/s), 194725 word types, 10000 tags
2019-04-08 07:31:33,546 : INFO : PROGRESS: at example #20000, processed 4206669 words (702698/s), 279127 word types, 20000 tags
2019-04-08 07:31:36,563 : INFO : PROGRESS: at example #30000, processed 6310699 words (697702/s), 340949 word types, 30000 tags
2019-04-08 07:31:39,628 : INFO : PROGRESS: at example #40000, processed 8434291 words (692997/s), 393353 word types, 40000 tags
2019-04-08 07:31:42,676 : INFO : PROGRESS: at example #50000, processed 10542423 words (691772/s), 437816 word types, 50000 tags
2019-04-08 07:31:45,714 : INFO : PROGRESS: at example #60000, processed 12652322 words (694688/s), 477648 word types, 60000 tags
2019-04-08 07:31:48,743 : INFO : PROGRESS: at example #

2019-04-08 07:33:45,941 : INFO : EPOCH 1 - PROGRESS: at 17.01% examples, 400865 words/s, in_qsize 7, out_qsize 0
2019-04-08 07:33:46,954 : INFO : EPOCH 1 - PROGRESS: at 17.63% examples, 400829 words/s, in_qsize 7, out_qsize 0
2019-04-08 07:33:47,957 : INFO : EPOCH 1 - PROGRESS: at 18.27% examples, 400894 words/s, in_qsize 7, out_qsize 0
2019-04-08 07:33:48,959 : INFO : EPOCH 1 - PROGRESS: at 18.91% examples, 401254 words/s, in_qsize 6, out_qsize 0
2019-04-08 07:33:49,960 : INFO : EPOCH 1 - PROGRESS: at 19.54% examples, 401338 words/s, in_qsize 7, out_qsize 0
2019-04-08 07:33:50,997 : INFO : EPOCH 1 - PROGRESS: at 20.19% examples, 401238 words/s, in_qsize 7, out_qsize 0
2019-04-08 07:33:52,005 : INFO : EPOCH 1 - PROGRESS: at 20.83% examples, 401532 words/s, in_qsize 7, out_qsize 0
2019-04-08 07:33:53,013 : INFO : EPOCH 1 - PROGRESS: at 21.45% examples, 401519 words/s, in_qsize 7, out_qsize 0
2019-04-08 07:33:54,015 : INFO : EPOCH 1 - PROGRESS: at 22.07% examples, 401580 words/s, in_qsiz

2019-04-08 07:34:59,863 : INFO : EPOCH 1 - PROGRESS: at 63.10% examples, 400949 words/s, in_qsize 7, out_qsize 0
2019-04-08 07:35:00,879 : INFO : EPOCH 1 - PROGRESS: at 63.74% examples, 401013 words/s, in_qsize 7, out_qsize 0
2019-04-08 07:35:01,879 : INFO : EPOCH 1 - PROGRESS: at 64.39% examples, 401132 words/s, in_qsize 7, out_qsize 0
2019-04-08 07:35:02,893 : INFO : EPOCH 1 - PROGRESS: at 65.02% examples, 401108 words/s, in_qsize 7, out_qsize 0
2019-04-08 07:35:03,901 : INFO : EPOCH 1 - PROGRESS: at 65.66% examples, 401191 words/s, in_qsize 7, out_qsize 0
2019-04-08 07:35:04,908 : INFO : EPOCH 1 - PROGRESS: at 66.27% examples, 401105 words/s, in_qsize 7, out_qsize 0
2019-04-08 07:35:05,916 : INFO : EPOCH 1 - PROGRESS: at 66.90% examples, 401106 words/s, in_qsize 8, out_qsize 0
2019-04-08 07:35:06,919 : INFO : EPOCH 1 - PROGRESS: at 67.53% examples, 401132 words/s, in_qsize 7, out_qsize 0
2019-04-08 07:35:07,933 : INFO : EPOCH 1 - PROGRESS: at 68.14% examples, 401103 words/s, in_qsiz

2019-04-08 07:36:10,190 : INFO : EPOCH 2 - PROGRESS: at 6.95% examples, 400827 words/s, in_qsize 7, out_qsize 0
2019-04-08 07:36:11,206 : INFO : EPOCH 2 - PROGRESS: at 7.58% examples, 401360 words/s, in_qsize 7, out_qsize 0
2019-04-08 07:36:12,212 : INFO : EPOCH 2 - PROGRESS: at 8.21% examples, 401368 words/s, in_qsize 7, out_qsize 0
2019-04-08 07:36:13,213 : INFO : EPOCH 2 - PROGRESS: at 8.86% examples, 402146 words/s, in_qsize 7, out_qsize 0
2019-04-08 07:36:14,226 : INFO : EPOCH 2 - PROGRESS: at 9.49% examples, 401940 words/s, in_qsize 7, out_qsize 0
2019-04-08 07:36:15,239 : INFO : EPOCH 2 - PROGRESS: at 10.13% examples, 401792 words/s, in_qsize 7, out_qsize 0
2019-04-08 07:36:16,249 : INFO : EPOCH 2 - PROGRESS: at 10.76% examples, 402186 words/s, in_qsize 7, out_qsize 0
2019-04-08 07:36:17,260 : INFO : EPOCH 2 - PROGRESS: at 11.38% examples, 402074 words/s, in_qsize 7, out_qsize 0
2019-04-08 07:36:18,272 : INFO : EPOCH 2 - PROGRESS: at 12.00% examples, 401891 words/s, in_qsize 7, 

2019-04-08 07:37:24,038 : INFO : EPOCH 2 - PROGRESS: at 53.08% examples, 401547 words/s, in_qsize 7, out_qsize 0
2019-04-08 07:37:25,047 : INFO : EPOCH 2 - PROGRESS: at 53.70% examples, 401534 words/s, in_qsize 8, out_qsize 0
2019-04-08 07:37:26,086 : INFO : EPOCH 2 - PROGRESS: at 54.34% examples, 401492 words/s, in_qsize 7, out_qsize 0
2019-04-08 07:37:27,104 : INFO : EPOCH 2 - PROGRESS: at 54.97% examples, 401445 words/s, in_qsize 8, out_qsize 0
2019-04-08 07:37:28,122 : INFO : EPOCH 2 - PROGRESS: at 55.60% examples, 401388 words/s, in_qsize 8, out_qsize 0
2019-04-08 07:37:29,144 : INFO : EPOCH 2 - PROGRESS: at 56.23% examples, 401325 words/s, in_qsize 7, out_qsize 0
2019-04-08 07:37:30,168 : INFO : EPOCH 2 - PROGRESS: at 56.87% examples, 401358 words/s, in_qsize 7, out_qsize 0
2019-04-08 07:37:31,171 : INFO : EPOCH 2 - PROGRESS: at 57.48% examples, 401273 words/s, in_qsize 8, out_qsize 0
2019-04-08 07:37:32,175 : INFO : EPOCH 2 - PROGRESS: at 58.12% examples, 401296 words/s, in_qsiz

...

2019-04-08 07:47:02,625 : INFO : EPOCH 6 - PROGRESS: at 14.08% examples, 405318 words/s, in_qsize 7, out_qsize 0
2019-04-08 07:47:03,631 : INFO : EPOCH 6 - PROGRESS: at 14.70% examples, 405176 words/s, in_qsize 7, out_qsize 0
2019-04-08 07:47:04,631 : INFO : EPOCH 6 - PROGRESS: at 15.33% examples, 404803 words/s, in_qsize 7, out_qsize 0
2019-04-08 07:47:05,647 : INFO : EPOCH 6 - PROGRESS: at 15.96% examples, 404535 words/s, in_qsize 7, out_qsize 0
2019-04-08 07:47:06,651 : INFO : EPOCH 6 - PROGRESS: at 16.57% examples, 404445 words/s, in_qsize 7, out_qsize 0
2019-04-08 07:47:07,671 : INFO : EPOCH 6 - PROGRESS: at 17.22% examples, 404507 words/s, in_qsize 8, out_qsize 0
2019-04-08 07:47:08,678 : INFO : EPOCH 6 - PROGRESS: at 17.85% examples, 404427 words/s, in_qsize 7, out_qsize 0
2019-04-08 07:47:09,688 : INFO : EPOCH 6 - PROGRESS: at 18.48% examples, 404258 words/s, in_qsize 7, out_qsize 0
2019-04-08 07:47:10,694 : INFO : EPOCH 6 - PROGRESS: at 19.11% examples, 404154 words/s, in_qsiz

2019-04-08 07:48:16,584 : INFO : EPOCH 6 - PROGRESS: at 60.26% examples, 402352 words/s, in_qsize 7, out_qsize 1
2019-04-08 07:48:17,584 : INFO : EPOCH 6 - PROGRESS: at 60.89% examples, 402367 words/s, in_qsize 7, out_qsize 0
2019-04-08 07:48:18,602 : INFO : EPOCH 6 - PROGRESS: at 61.52% examples, 402398 words/s, in_qsize 7, out_qsize 0
2019-04-08 07:48:19,611 : INFO : EPOCH 6 - PROGRESS: at 62.15% examples, 402378 words/s, in_qsize 8, out_qsize 0
2019-04-08 07:48:20,622 : INFO : EPOCH 6 - PROGRESS: at 62.78% examples, 402344 words/s, in_qsize 7, out_qsize 0
2019-04-08 07:48:21,627 : INFO : EPOCH 6 - PROGRESS: at 63.40% examples, 402349 words/s, in_qsize 7, out_qsize 0
2019-04-08 07:48:22,630 : INFO : EPOCH 6 - PROGRESS: at 64.03% examples, 402351 words/s, in_qsize 7, out_qsize 0
2019-04-08 07:48:23,637 : INFO : EPOCH 6 - PROGRESS: at 64.65% examples, 402339 words/s, in_qsize 7, out_qsize 0
2019-04-08 07:48:24,645 : INFO : EPOCH 6 - PROGRESS: at 65.28% examples, 402325 words/s, in_qsiz

2019-04-08 07:49:27,227 : INFO : EPOCH 7 - PROGRESS: at 4.45% examples, 405201 words/s, in_qsize 6, out_qsize 0
2019-04-08 07:49:28,241 : INFO : EPOCH 7 - PROGRESS: at 5.08% examples, 404267 words/s, in_qsize 7, out_qsize 0
2019-04-08 07:49:29,246 : INFO : EPOCH 7 - PROGRESS: at 5.71% examples, 404077 words/s, in_qsize 7, out_qsize 0
2019-04-08 07:49:30,270 : INFO : EPOCH 7 - PROGRESS: at 6.36% examples, 404110 words/s, in_qsize 7, out_qsize 0
2019-04-08 07:49:31,271 : INFO : EPOCH 7 - PROGRESS: at 6.99% examples, 404152 words/s, in_qsize 7, out_qsize 0
2019-04-08 07:49:32,288 : INFO : EPOCH 7 - PROGRESS: at 7.61% examples, 403612 words/s, in_qsize 7, out_qsize 0
2019-04-08 07:49:33,308 : INFO : EPOCH 7 - PROGRESS: at 8.25% examples, 403708 words/s, in_qsize 8, out_qsize 0
2019-04-08 07:49:34,310 : INFO : EPOCH 7 - PROGRESS: at 8.89% examples, 403657 words/s, in_qsize 8, out_qsize 0
2019-04-08 07:49:35,312 : INFO : EPOCH 7 - PROGRESS: at 9.51% examples, 403038 words/s, in_qsize 8, out_

2019-04-08 07:50:41,416 : INFO : EPOCH 7 - PROGRESS: at 50.77% examples, 401867 words/s, in_qsize 8, out_qsize 0
2019-04-08 07:50:42,443 : INFO : EPOCH 7 - PROGRESS: at 51.42% examples, 401879 words/s, in_qsize 7, out_qsize 0
2019-04-08 07:50:43,468 : INFO : EPOCH 7 - PROGRESS: at 52.05% examples, 401794 words/s, in_qsize 8, out_qsize 0
2019-04-08 07:50:44,473 : INFO : EPOCH 7 - PROGRESS: at 52.68% examples, 401802 words/s, in_qsize 7, out_qsize 0
2019-04-08 07:50:45,489 : INFO : EPOCH 7 - PROGRESS: at 53.33% examples, 401865 words/s, in_qsize 7, out_qsize 0
2019-04-08 07:50:46,512 : INFO : EPOCH 7 - PROGRESS: at 53.97% examples, 401895 words/s, in_qsize 7, out_qsize 0
2019-04-08 07:50:47,522 : INFO : EPOCH 7 - PROGRESS: at 54.60% examples, 401875 words/s, in_qsize 8, out_qsize 0
2019-04-08 07:50:48,556 : INFO : EPOCH 7 - PROGRESS: at 55.24% examples, 401855 words/s, in_qsize 7, out_qsize 0
2019-04-08 07:50:49,563 : INFO : EPOCH 7 - PROGRESS: at 55.87% examples, 401843 words/s, in_qsiz

2019-04-08 07:51:55,514 : INFO : EPOCH 7 - PROGRESS: at 96.98% examples, 401796 words/s, in_qsize 7, out_qsize 0
2019-04-08 07:51:56,525 : INFO : EPOCH 7 - PROGRESS: at 97.60% examples, 401782 words/s, in_qsize 7, out_qsize 0
2019-04-08 07:51:57,532 : INFO : EPOCH 7 - PROGRESS: at 98.23% examples, 401776 words/s, in_qsize 7, out_qsize 0
2019-04-08 07:51:58,551 : INFO : EPOCH 7 - PROGRESS: at 98.86% examples, 401745 words/s, in_qsize 7, out_qsize 0
2019-04-08 07:51:59,561 : INFO : EPOCH 7 - PROGRESS: at 99.47% examples, 401676 words/s, in_qsize 6, out_qsize 1
2019-04-08 07:52:00,237 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-04-08 07:52:00,244 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-04-08 07:52:00,247 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-04-08 07:52:00,249 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-04-08 07:52:00,249 : INFO : EPOCH - 7 : training on 69209625 raw w

In [8]:
doc2vec_model.infer_vector(['эти', 'сладости', 'признали', 'полезными', 'для', 'здоровья', 'многие', 'люди', 'безгранично', 'любят', 'сладости', 'ведь', 'сладкое', 'способствует', 'повышению', 'гормона', 'серотонина', 'который', 'вызывает', 'ощущение', 'радости', 'хоть', 'кратковременной', 'однако', 'за', 'приятным', 'вкусом', 'лакомств', 'таится', 'немало', 'побочных', 'эффектов', 'склонность', 'высыпаниям', 'лишний', 'вес', 'повышение', 'сахара', 'крови', 'но', 'есть', 'лакомство', 'которого', 'нет', 'перечисленных', 'побочных', 'эффектов', 'горький', 'шоколад', 'он', 'содержит', 'немало', 'полезных', 'витаминов', 'веществ', 'частности', 'нем', 'есть', 'd2', 'такой', 'шоколад', 'уменьшает', 'воспалительные', 'процессы', 'организме', 'даже', 'продлевает', 'жизнь', 'черном', 'шоколаде', 'практически', 'нет', 'сахара', 'содержится', 'много', 'какао', 'поэтому', 'он', 'не', 'вредит', 'здоровью', 'приносит', 'пользу', 'витамины', 'группы', 'которые', 'он', 'содержит', 'улучшают', 'функционирование', 'щитовидной', 'железы', 'благоприятно', 'влияют', 'на', 'свертываемость', 'крови', 'для', 'тех', 'кто', 'хочет', 'есть', 'сладкое', 'без', 'последствий', 'горький', 'темный', 'шоколад', 'настоящее', 'спасение', 'главное', 'привыкнуть', 'его', 'вкусу', 'ведь', 'он', 'не', 'будет', 'таким', 'же', 'сладким', 'как', 'вредный', 'пончик', 'усыпанный', 'сахарной', 'пудрой', 'необходимо', 'внимательно', 'читать', 'этикетку', 'плитки', 'шоколада', 'первым', 'списке', 'ингредиентов', 'должно', 'быть', 'какао', 'сахара', 'вообще', 'не', 'должно', 'быть', 'или', 'небольшом', 'количестве', 'таком', 'случае', 'сахар', 'будет', 'стоять', 'на', 'последних', 'местах', 'перечня', 'составляющих', 'ингредиентов', ').', 'еще', 'по', 'теме'])

array([ 0.09846976, -0.15566477,  0.0835371 , -0.34671313, -0.3453695 ,
        0.02082629,  0.1194124 , -0.09499438,  0.30113062,  0.1985944 ,
       -0.09428383, -0.8362487 ,  0.17740779,  0.34675512, -0.13062663,
        0.93127245,  0.55507904,  0.35757565, -0.30729115, -0.05034177,
       -0.3552248 ,  0.31838176, -0.37761167,  0.22893813,  0.16101088,
        0.41763857, -0.23043008,  0.045691  , -0.30314237,  0.07790605,
        0.29989156,  0.24590378,  0.00435978, -0.5301655 , -0.06675083,
        0.30852473,  0.13580568,  0.48323646,  0.36399412, -0.39461088,
        0.40679604,  0.32074594, -0.5285727 , -0.37357864, -0.13340975,
       -0.06979224, -0.08570327, -0.08693221, -0.15238969, -0.15636621,
       -0.23586361, -0.66407067,  0.32110983, -0.12165106, -0.31108493,
       -0.22356589,  0.03017702, -0.67679644, -0.6012838 ,  0.48458812,
        0.6635858 , -0.60342044, -0.14325307,  0.1659576 ], dtype=float32)

In [9]:
doc2vec_model.infer_vector(items_df.iloc[0]['content'])

array([-0.00609336, -0.00067727, -0.00141699,  0.00605677, -0.00153788,
       -0.00563633,  0.00106861, -0.00531802, -0.00518857,  0.00510948,
       -0.00506645, -0.00392989,  0.00220281,  0.00321719, -0.00063542,
       -0.00726005, -0.00290134, -0.00693984,  0.00663828,  0.00146688,
        0.00584508, -0.00280486, -0.00529323,  0.00696235, -0.00251812,
       -0.00050771,  0.0025838 , -0.00560743, -0.00529442, -0.00555857,
       -0.00727661,  0.0033517 ,  0.00415892,  0.00655977, -0.0054093 ,
        0.00554865,  0.00441382,  0.00416877, -0.00183475, -0.00094635,
       -0.0069047 , -0.00071202, -0.00714984,  0.00081154,  0.00472731,
        0.00620023, -0.0069321 ,  0.00273122, -0.0035155 ,  0.00587238,
       -0.00476216,  0.00357696, -0.00182186,  0.00601957, -0.00760765,
       -0.00279653, -0.00245862, -0.00019485,  0.00215963, -0.0048666 ,
       -0.0011376 , -0.00201245, -0.0055263 ,  0.00054454], dtype=float32)

In [10]:
from utils.prepare_data import tokenize

In [11]:
doc2vec_model.infer_vector(tokenize(items_df.iloc[0]['content']))

array([-0.04293876,  0.99834484, -0.54241633,  0.5167112 ,  0.13665412,
       -0.87072057,  0.03146183, -0.33654037,  0.1734029 , -0.6990801 ,
        0.6131044 , -0.03982187, -0.11985139, -0.01893762,  0.14787437,
       -0.7997196 ,  1.2021552 ,  0.35512862, -0.27570945, -0.93679637,
       -0.6604718 ,  0.17823797, -0.5252819 ,  0.05099907, -0.16495462,
        0.48208675,  0.04480349,  0.76150876,  0.44993123, -0.26707917,
        0.27887735, -0.04359797, -0.55285203, -0.05305161, -0.15412776,
        0.08304303, -0.06292044, -0.59214216,  1.4836172 ,  0.42414317,
        0.8716312 ,  1.0487088 , -0.3236487 ,  0.43521452, -0.07293334,
        0.28845719,  0.30240545,  1.180163  , -0.26621458, -0.75598544,
       -0.04276308, -0.5640192 , -0.35504884,  1.3319682 , -0.25979108,
        0.0101698 , -0.47128776,  0.20270239, -0.6595978 ,  0.4515462 ,
       -0.60161304, -0.59595835, -0.3916927 ,  0.08281218], dtype=float32)

In [12]:
def process_item(item_id):
    item = items_df.iloc[item_id]
    return np.concatenate((doc2vec_model.infer_vector(tokenize('{} {}'.format(item["title"], item["content"]))), item['image']))

In [13]:
def get_random_responses_generator(users_df):
    while True:
        user_id, items, ratings = users_df.iloc[np.random.randint(0, users_df.shape[0])]
        rnd_i = np.random.randint(0, len(items))
        item_id = items[rnd_i]
        rating = ratings[rnd_i]
        yield user_id, process_item(item_id), rating

In [14]:
def get_responses_generator(users_df):
    for _, (user_id, items, ratings) in users_df.iterrows():
        for item_id, rating in zip(items, ratings):
            yield user_id, process_item(item_id), rating

In [15]:
def batch_generator(responses_generator, batch_size=32):
    user_ids, user_items, user_ratings = [], [], []
    i = -1
    for user_id, user_item, user_rating in responses_generator:
        user_ids.append(user_id)
        user_items.append(user_item)
        user_ratings.append(user_rating)
        if i % batch_size == 0:
            yield tuple(map(np.array, [user_ids, user_items, user_ratings]))
            user_ids, user_items, user_ratings = [], [], []
        i += 1

In [16]:
def reset_tf_session():
    curr_session = tf.get_default_session()
    # close current session
    if curr_session is not None:
        curr_session.close()
    # reset graph
    tf.reset_default_graph()
    # create new session
#     config = tf.ConfigProto()
#     config.gpu_options.allow_growth = True
#     config.allow_soft_placement=True
#     config.log_device_placement=True
#     s = tf.InteractiveSession(config=config)
#     return s

In [28]:
class MLP():
    def __init__(self, doc2vec_model, n_users, user_embedding_size, item_embedding_size, hidden_size, n_layers, sess, log_file):
        self.doc2vec_model = doc2vec_model
        self.n_users = n_users
        self.user_embedding_size = user_embedding_size
        self.item_embedding_size = item_embedding_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.sess = sess
        self.log_file = open(log_file, 'w')
#         self.n_items = n_items

    def define_graph(self, learning_rate):
        self.user_id = tf.placeholder(tf.int32, shape=[None], name='user_id')
        self.user_embeddings = tf.get_variable(
            'user_embeddings',
            shape=[self.n_users, self.user_embedding_size],
            dtype=tf.float32
        )
        print(self.user_id)
        print(self.user_embeddings)
        user_embedding = tf.nn.embedding_lookup(
            self.user_embeddings, self.user_id)
        print(user_embedding)
        self.item_embedding = tf.placeholder(
            tf.float32, shape=[None, self.item_embedding_size])
        print(self.item_embedding)
        self.y = tf.placeholder(tf.float32, name='y')
        

        layer = tf.concat([user_embedding, self.item_embedding], axis=1)
        print(layer)
        for i in np.arange(self.n_layers):
            layer = tf.layers.dense(
                layer, self.hidden_size, tf.nn.elu, kernel_initializer=tf.random_normal_initializer)
        self.p = tf.layers.dense(layer, 1, tf.nn.sigmoid, kernel_initializer=tf.random_normal_initializer)
        print(self.p)
        self.loss = -tf.reduce_sum(self.y * tf.log(self.p + 1e-10) + (1 - self.y) * tf.log(1 - self.p + 1e-10))
        print(self.loss)
        self.optimizer = tf.train.AdagradOptimizer(learning_rate).minimize(self.loss)
    
    def train(self, train_df, steps):
        train_ratings = get_random_responses_generator(train_df)
        loss = []
        start_time = time.time()
        for it, (batch_user, batch_item, batch_rating) in enumerate(batch_generator(train_ratings)):
            if (it + 1) % steps == 0:
                break
#             batch_rating[batch_rating == 0] = -1
            _, l = self.sess.run((self.optimizer, self.loss),
                                feed_dict={self.user_id: batch_user, self.item_embedding: batch_item, self.y: batch_rating})
#                 print("Iteration: %04d; cost= %.9f" % (it, np.mean(loss)))
            loss.append(l)
            if it % 1000 == 0:
                print(f'Iteration: {it}, loss: {np.mean(loss)/32}, elapsed time: {time.time() - start_time}')
                print(f'Iteration: {it}, loss: {np.mean(loss)/32}, elapsed time: {time.time() - start_time}', file=self.log_file, flush=True)
                start_time = time.time()
#                 print(self.predict(batch_user, batch_item))
#                 print(batch_rating)
                

    def execute(self, train_df, epochs, epoch_steps, learning_rate):
#         self.sess = reset_tf_session()
        self.define_graph(learning_rate)
        init = tf.global_variables_initializer()
        self.sess.run(init)
        
        for epoch in np.arange(epochs)+1:
            self.train(train_df, epoch_steps)
            print("Epoch: %04d; " % (epoch), end='')
#             self.test(test_df)
            
    
    def test(self, test_df):
        loss = []
        for user_id, item, rating in tqdm(batch_generator(get_responses_generator(test_df))):
            p = self.predict(user_id, item)
            loss.append(rating * np.log(p) + (1 - rating) * tf.log(1 - p))
        return np.mean(loss)
    
    
    def predict(self, user_id, item):
        return self.sess.run([self.p], feed_dict={self.user_id: user_id, self.item_embedding: item})[0]
    
    def save(self, path):
        saver = tf.train.Saver()
        saver.save(self.sess, path)

In [29]:
epoch_steps = sum(map(len, train_df['userItems'].values))
epoch_steps = epoch_steps // 32
epoch_steps

1693435

In [30]:
96+64

160

In [31]:
!pwd

/data/home/Xetd71/Content-based-Neural-Recommender-Systems/models/mlp


In [34]:
doc2vec_model.save('/data/home/Xetd71/doc2vec_model/doc2vec_model')

2019-04-08 11:58:35,639 : INFO : saving Doc2Vec object under /data/home/Xetd71/doc2vec_model/doc2vec_model, separately None
2019-04-08 11:58:35,641 : INFO : storing np array 'syn1neg' to /data/home/Xetd71/doc2vec_model/doc2vec_model.trainables.syn1neg.npy
2019-04-08 11:58:35,698 : INFO : storing np array 'vectors' to /data/home/Xetd71/doc2vec_model/doc2vec_model.wv.vectors.npy
2019-04-08 11:58:35,753 : INFO : storing np array 'vectors_docs' to /data/home/Xetd71/doc2vec_model/doc2vec_model.docvecs.vectors_docs.npy
2019-04-08 11:58:36,582 : INFO : saved /data/home/Xetd71/doc2vec_model/doc2vec_model


In [32]:
reset_tf_session()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
model = MLP(doc2vec_model, train_df.shape[0], 64, 160, 128, 1, sess, '/data/home/Xetd71/model1/log.txt')
model.execute(train_df, 5, epoch_steps, 0.01)
model.save('/data/home/Xetd71/model1/model')

Tensor("user_id:0", shape=(?,), dtype=int32)
<tf.Variable 'user_embeddings:0' shape=(42977, 64) dtype=float32_ref>
Tensor("embedding_lookup/Identity:0", shape=(?, 64), dtype=float32)
Tensor("Placeholder:0", shape=(?, 160), dtype=float32)
Tensor("concat:0", shape=(?, 224), dtype=float32)
Tensor("dense_1/Sigmoid:0", shape=(?, 1), dtype=float32)
Tensor("Neg:0", shape=(), dtype=float32)
Iteration: 0, loss: 1.43911612033844, elapsed time: 0.06722497940063477
Iteration: 1000, loss: 87.47600555419922, elapsed time: 98.92887425422668
Iteration: 2000, loss: 77.55045318603516, elapsed time: 98.77329063415527
Iteration: 3000, loss: 71.56993865966797, elapsed time: 98.79427194595337
Iteration: 4000, loss: 67.39175415039062, elapsed time: 98.59812879562378
Iteration: 5000, loss: 64.11557006835938, elapsed time: 98.80614590644836
Iteration: 6000, loss: 61.51530838012695, elapsed time: 98.47988271713257
Iteration: 7000, loss: 59.50559616088867, elapsed time: 98.51895403862
Iteration: 8000, loss: 57.8

Iteration: 104000, loss: 27.795713424682617, elapsed time: 94.31869292259216
Iteration: 105000, loss: 27.711223602294922, elapsed time: 94.1426191329956
Iteration: 106000, loss: 27.622602462768555, elapsed time: 93.85152053833008
Iteration: 107000, loss: 27.539011001586914, elapsed time: 94.17080497741699
Iteration: 108000, loss: 27.455711364746094, elapsed time: 94.02540111541748
Iteration: 109000, loss: 27.3741455078125, elapsed time: 94.02606987953186
Iteration: 110000, loss: 27.295795440673828, elapsed time: 93.72929954528809
Iteration: 111000, loss: 27.215362548828125, elapsed time: 94.00020718574524
Iteration: 112000, loss: 27.137187957763672, elapsed time: 94.14265704154968
Iteration: 113000, loss: 27.05817413330078, elapsed time: 94.0491042137146
Iteration: 114000, loss: 26.980045318603516, elapsed time: 93.85206770896912
Iteration: 115000, loss: 26.9031982421875, elapsed time: 94.1404025554657
Iteration: 116000, loss: 26.829145431518555, elapsed time: 94.12587380409241
Iterati

KeyboardInterrupt: 

In [33]:
model.save('/data/home/Xetd71/model1/model')

In [ ]:
reset_tf_session()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
model2 = MLP(doc2vec_model, train_df.shape[0], 64, 160, 128, 2, sess, '/data/home/Xetd71/model2/log.txt')
model2.execute(train_df, 10, epoch_steps, 0.01)
model.save('/data/home/Xetd71/model2/model')

In [ ]:
reset_tf_session()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
model3 = MLP(doc2vec_model, train_df.shape[0], 64, 160, 128, 3, sess, '/data/home/Xetd71/model3/log.txt')
model3.execute(train_df, 10, epoch_steps, 0.01)
model.save('/data/home/Xetd71/model3/model')

In [ ]:
import tensorflow as tf
import time
import numpy as np
import random

from utils.evaluation.RankingMetrics import *

class MLP():
    def __init__(self, sess, num_user, num_item, learning_rate=0.5, reg_rate=0.001, epoch=500, batch_size=256,
                 verbose=False, T=5, display_step=1000):
        self.learning_rate = learning_rate
        self.epochs = epoch
        self.batch_size = batch_size
        self.reg_rate = reg_rate
        self.sess = sess
        self.num_user = num_user
        self.num_item = num_item
        self.verbose = verbose
        self.T = T
        self.display_step = display_step
        print("You are running MLP.")

    def build_network(self, num_factor_mlp=10, hidden_dimension=10, num_neg_sample=2):
        self.num_neg_sample = num_neg_sample
        self.user_id = tf.placeholder(dtype=tf.int32, shape=[None], name='user_id')
        self.item_id = tf.placeholder(dtype=tf.int32, shape=[None], name='item_id')
        self.y = tf.placeholder(dtype=tf.float32, shape=[None], name='y')

        self.mlp_P = tf.Variable(tf.random_normal([self.num_user, num_factor_mlp]), dtype=tf.float32)
        self.mlp_Q = tf.Variable(tf.random_normal([self.num_item, num_factor_mlp]), dtype=tf.float32)

        mlp_user_latent_factor = tf.nn.embedding_lookup(self.mlp_P, self.user_id)
        mlp_item_latent_factor = tf.nn.embedding_lookup(self.mlp_Q, self.item_id)

        layer_1 = tf.layers.dense(inputs=tf.concat([mlp_item_latent_factor, mlp_user_latent_factor], axis=1),
                                  units=num_factor_mlp * 2, kernel_initializer=tf.random_normal_initializer,
                                  activation=tf.nn.relu,
                                  kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=self.reg_rate))
        layer_2 = tf.layers.dense(inputs=layer_1, units=hidden_dimension * 2, activation=tf.nn.relu,
                                  kernel_initializer=tf.random_normal_initializer,
                                  kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=self.reg_rate))
        MLP = tf.layers.dense(inputs=layer_2, units=hidden_dimension, activation=tf.nn.relu,
                              kernel_initializer=tf.random_normal_initializer,
                              kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=self.reg_rate))

        self.pred_y = tf.nn.sigmoid(tf.reduce_sum(MLP, axis=1))

        # self.pred_y = tf.layers.dense(inputs=MLP, units=1, activation=tf.sigmoid)

        self.loss = - tf.reduce_sum(
            self.y * tf.log(self.pred_y + 1e-10) + (1 - self.y) * tf.log(1 - self.pred_y + 1e-10)) \
                    + tf.losses.get_regularization_loss() + self.reg_rate * (
        tf.nn.l2_loss(self.mlp_P) + tf.nn.l2_loss(self.mlp_Q))

        self.optimizer = tf.train.AdagradOptimizer(self.learning_rate).minimize(self.loss)

        return self

    def prepare_data(self, train_data, test_data):
        '''
        You must prepare the data before train and test the model
        :param train_data:
        :param test_data:
        :return:
        '''
        t = train_data.tocoo()
        self.user = list(t.row.reshape(-1))
        self.item = list(t.col.reshape(-1))
        self.label = list(t.data)
        self.test_data = test_data

        self.neg_items = self._get_neg_items(train_data.tocsr())
        self.test_users = set([u for u in self.test_data.keys() if len(self.test_data[u]) > 0])

        print("data preparation finished.")
        return self

    def train(self):

        item_temp = self.item[:]
        user_temp = self.user[:]
        labels_temp = self.label[:]

        user_append = []
        item_append = []
        values_append = []
        for u in self.user:
            list_of_random_items = random.sample(self.neg_items[u], self.num_neg_sample)
            user_append += [u] * self.num_neg_sample
            item_append += list_of_random_items
            values_append += [0] * self.num_neg_sample

        item_temp += item_append
        user_temp += user_append
        labels_temp += values_append

        self.num_training = len(item_temp)
        self.total_batch = int(self.num_training / self.batch_size)
        # print(self.total_batch)
        idxs = np.random.permutation(self.num_training)  # shuffled ordering
        user_random = list(np.array(user_temp)[idxs])
        item_random = list(np.array(item_temp)[idxs])
        labels_random = list(np.array(labels_temp)[idxs])

        # train
        for i in range(self.total_batch):
            start_time = time.time()
            batch_user = user_random[i * self.batch_size:(i + 1) * self.batch_size]
            batch_item = item_random[i * self.batch_size:(i + 1) * self.batch_size]
            batch_label = labels_random[i * self.batch_size:(i + 1) * self.batch_size]

            _, loss = self.sess.run((self.optimizer, self.loss),
                                    feed_dict={self.user_id: batch_user, self.item_id: batch_item, self.y: batch_label})

            if i % self.display_step == 0:
                if self.verbose:
                    print("Index: %04d; cost= %.9f" % (i + 1, np.mean(loss)))
                    print("one iteration: %s seconds." % (time.time() - start_time))

    def test(self):
        evaluate(self)

    def execute(self, train_data, test_data):

        self.prepare_data(train_data, test_data)

        init = tf.global_variables_initializer()
        self.sess.run(init)

        for epoch in range(self.epochs):
            self.train()
            if (epoch) % self.T == 0:
                print("Epoch: %04d; " % (epoch), end='')
                self.test()

    def save(self, path):
        saver = tf.train.Saver()
        saver.save(self.sess, path)

    def predict(self, user_id, item_id):
        return self.sess.run([self.pred_y], feed_dict={self.user_id: user_id, self.item_id: item_id})[0]

    def _get_neg_items(self, data):
        all_items = set(np.arange(self.num_item))
        neg_items = {}
        for u in range(self.num_user):
            neg_items[u] = list(all_items - set(data.getrow(u).nonzero()[1]))

        return neg_items